# Use reActAgent to perform complicated task


In this example, we create a *reactAgent* to perform a more complex task: writing a markdown solution for a simple physics problem. To solve the problem, the agent needs to:
- Search the internet to obtain the exact values of the required results.
- Generate the solution to the problem.
- Write the solution to a file in markdown format

Let's set the model configuration at first

In [1]:
MODEL_CONFIGURATION = {
    "config_name": "gpt-4",
    "model_type": "openai_chat",
    "model_name": "gpt-4",
    "api_key": "YOUR API",
}
SEARCH_ENGINE_API = "YOUR SEARCH ENGINE API"

*AgentScope* offers a series of services, including one that can enable the agent to search the internet and obtain the exact values required for the problem.

In [2]:
from agentscope.service import ServiceFactory
from agentscope.service import bing_search
tools = [
    ServiceFactory.get(bing_search, api_key=SEARCH_ENGINE_API, num_results=3),
]

Let's create a *ReActAgent* and specify the tools can be accessed by it.

In [3]:
from agentscope.agents import ReActAgent
import agentscope
agentscope.init(model_configs=MODEL_CONFIGURATION)
agent = ReActAgent(
    name="assistant",
    model_config_name="gpt-4",
    tools=tools,
    sys_prompt="You are a helpful assistant.",
    verbose=True,
)

2024-04-18 16:03:10.918 | INFO     | agentscope.models:read_model_configs:178 - Load configs for model wrapper: gpt-4
2024-04-18 16:03:10.933 | INFO     | agentscope.utils.monitor:_create_monitor_table:343 - Init [monitor_metrics] as the monitor table
2024-04-18 16:03:10.934 | INFO     | agentscope.utils.monitor:_create_monitor_table:344 - Init [monitor_metrics_quota_exceeded] as the monitor trigger
2024-04-18 16:03:10.934 | INFO     | agentscope.utils.monitor:__init__:313 - SqliteMonitor initialization completed at [./runs\run_20240418-160308_9q4v9y\agentscope.db]
2024-04-18 16:03:10.936 | INFO     | agentscope.models.model:__init__:204 - Initialize model [gpt-4]


2024-04-18 16:03:11.251 | INFO     | agentscope.utils.monitor:register_budget:556 - set budget None to gpt-4
2024-04-18 16:03:11.255 | INFO     | agentscope.utils.monitor:register:364 - Register metric [gpt-4.cost] to SqliteMonitor with unit [dollor] and quota [None]
2024-04-18 16:03:11.262 | INFO     | agentscope.utils.monitor:register:364 - Register metric [gpt-4.prompt_tokens] to SqliteMonitor with unit [token] and quota [None]
2024-04-18 16:03:11.278 | INFO     | agentscope.utils.monitor:register:364 - Register metric [gpt-4.completion_tokens] to SqliteMonitor with unit [token] and quota [None]
2024-04-18 16:03:11.292 | INFO     | agentscope.utils.monitor:register:364 - Register metric [gpt-4.call_counter] to SqliteMonitor with unit [times] and quota [None]
2024-04-18 16:03:11.301 | INFO     | agentscope.utils.monitor:register:364 - Register metric [gpt-4.total_tokens] to SqliteMonitor with unit [token] and quota [None]


The initial prompt of the *reactAgent* can be viewed, which demonstrates how *AgentScope* automatically notifies the agent about the available tools and services that can be leveraged to complete the task.

In [4]:
print(agent.sys_prompt)

You are a helpful assistant.
The following tool functions are available in the format of
```
{index}. {function name}: {function description}
    {argument1 name} ({argument type}): {argument description}
    {argument2 name} ({argument type}): {argument description}
    ...
```

## Tool Functions:
1. bing_search: Search question in Bing Search API and return the searching results
	question (string): The search query string.

## What You Should Do:
1. First, analyze the current situation, and determine your goal.
2. Then, check if your goal is already achieved. If so, try to generate a response. Otherwise, think about how to achieve it with the help of provided tool functions.
3. Respond in the required format.

## Note:
1. Fully understand the tool functions and their arguments before using them.
2. You should decide if you need to use the tool functions, if not then return an empty list in "function" field.
3. Make sure the types and values of the arguments you provided to the tool f

Note how AgentScope automatically instructs the agent to utilize the provided tool in initial prompt. We can test the agent's capability to search online and obtain accurate results by asking them: 'Who has more average points per game during their career, LeBron James or Stephen Curry?'"

In [5]:
from agentscope.message import Msg
msg = Msg(
    name="user", 
    content="Who has more average points during his career, LeBron James or Stephen Curry?",
    role="user"
)

In [6]:
agent(msg)

##################### ITER 1, STEP 1: REASONING ######################
assistant: 
system: Your response is not a JSON object, and cannot be parsed by `json.loads` in parse function:
## Your Response:
[YOUR RESPONSE BEGIN]

[YOUR RESPONSE END]

## Error Information:
JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Analyze the reason, and re-correct your response in the correct format.
##################### ITER 2, STEP 1: REASONING ######################
assistant: {
    "thought": "The user wants to know who has more average points during his career, LeBron James or Stephen Curry. I don't have this information, so I will use the 'bing_search' function twice, once to find the average career points for LeBron James and once for Stephen Curry.",
    "speak": "Let me check this information for you.",
    "function": [
        {
            "name": "bing_search",
            "arguments": {
                "question": "LeBron James career average points"
            }
        },


{'id': '5fce2d0813aa48c1ae720957c957674a',
 'timestamp': '2024-04-18 16:03:40',
 'name': 'assistant',
 'content': {'thought': 'I found the results for both players. LeBron James has averaged 27.1 points during his career. On the other hand, Stephen Curry has averaged 24.8 points.',
  'speak': "LeBron James has a career average of 27.1 points per game, while Stephen Curry's career average is 24.8 points per game. Therefore, LeBron James has a higher career average than Stephen Curry.",
  'function': []},
 'role': 'assistant',
 'url': None}

Great, the statistics are correct. let's proceed to  enable the agent write file on your machine, you can write you own service following the protocal:

Run `!pip install pypandoc` to install required library.

In [7]:
from agentscope.service import ServiceResponse, ServiceExecStatus, retrieval
def write_md_file(latex_str:str, file_name:str) -> ServiceResponse:
    """
    Save a string in latex format to a Markdown file with the given file name.
    Args:
        latex_str (str): the string in latex format
        file_name (str): the name of the file to be created
    """
    try:
        import pypandoc as pdc
        pdc.convert_text(latex_str, 'md', 
                                     format='latex', 
                                     outputfile=file_name)
        output = f"Markdown file {file_name} has been created."
        status = ServiceExecStatus.SUCCESS
    except Exception as e:
        output = f"Error: {e}"
        status = ServiceExecStatus.ERROR
    return ServiceResponse(status, output)   
    

Note that it's important to provide clear documentation that explains your code, allowing the agent to understand how to effectively utilize your tool."

The ultimate question is 
> What is the kinetic energy of LeBron Jamees if he were running at the same speed Usain Bolt"achieved when setting the 100m world record? Write me the full reasoning and solution in Markdown format."

In [8]:
QUESTION = ("What is the kinetic energy of LeBron James"
            "if he were running at the same speed Usain Bolt" 
            "achieved when setting the 100m world record?"
            "Write me the full reasoning and solution in Markdown format.")

we create an ReAct Agent and power it with necessary tools

In [9]:
from agentscope.service import execute_python_code
tools = [
    ServiceFactory.get(bing_search, api_key=SEARCH_ENGINE_API, num_results=3),
    ServiceFactory.get(execute_python_code),
    ServiceFactory.get(write_md_file)
]
agent = ReActAgent(
    name="assistant",
    model_config_name="gpt-4",
    tools=tools,
    sys_prompt="You are a helpful assistant.",
    verbose=True,
)
msg = Msg(
    role="user",
    name="user",
    content=QUESTION
)

In [10]:
agent(msg)

{'id': '3e9a5a6830724aa2a34302edb8559d01',
 'timestamp': '2024-04-11 23:24:57',
 'name': 'assistant',
 'content': {'thought': 'The Markdown file with the full reasoning and calculation has been successfully created.',
  'speak': "The full reasoning and solution have been written into a Markdown file named 'kinetic_energy.md'.",
  'function': []},
 'role': 'assistant',
 'url': None}

Good it seems that the agent successfuly sovle the problem and output it as file *files/kinetic_energy.md*. You can check its solution yourself. 

---